In [11]:
import numpy as np
import matplotlib.pyplot as plt

import time
from copy import deepcopy

import cv2

from IPython.display import clear_output

In [2]:
%load_ext line_profiler

# Useful resources
[OpenCV tutorial](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html#playing-video-from-file)

# Define types for features

In [110]:
class Image(object):
    def __init__(self, img):
        self.img = img
        self.pixelsAccessed = []
    
    def getHSV(self, x, y):
        x = int(x)
        y = int(y)
        self.pixelsAccessed.append((x, y))
        rgb = self.img[y:y+1, x:x+1]
        hsv = cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV)
        return hsv[0, 0]
    
    def draw(self, img):
        color = (0, 0, 255)
        for p in self.pixelsAccessed:
            img[p[1], p[0]] = color

In [4]:
class Color(object):
    def __init__(self, minimum, maximum, colorspace='HSV'):
        if colorspace != 'HSV':
            raise Exception("Invalid colorspace, only HSV supported")
            
        self.min = minimum
        self.max = maximum
        
    def matches(self, color):
        # Check hue condition
        if self.min[0] > self.max[0]:
            # The hue range wraps past the edge
            if color[0] > self.max[0] and color[0] < self.min[0]:
                return False
        else:
            if color[0] > self.max[0] or color[0] < self.min[0]:
                return False
            
        # Check saturation and value conditions
        for i in (1, 2):
            if color[i] > self.max[i] or color[i] < self.min[i]:
                return False
        
        return True

In [138]:
class Circle2D(object):
    def __init__(self, center, radius, colorRange):
        self.center = np.array(center)
        self.radius = radius
        self.color  = colorRange
        
        # TODO: this should be a scalar with more reasonable behavior
        self.confidence = True
    
    def draw(self, img):
        """
        Taking in a numpy array representing an OpenCV image,
        draws a visual debugging indication of this feature
        """
        drawColor = (128, 255, 0) if self.confidence else (0, 0, 255)
        cv2.circle(img, tuple(self.center), self.radius, drawColor, thickness=5)
        cv2.circle(img, tuple(self.center), 5, drawColor, thickness=-1)
        
    def refine(self, image, verbose=False, searchRange = 200):
        """
        Taking in an Image object, updates this feature to better align
        with the observed pixels.
        
        searchRange represents the maximum pixel distance the target
            object could have moved.
        """
        
        def matchesAt(x, y):
            return self.color.matches(image.getHSV(x, y))

            
        
        if matchesAt(*self.center):
            self.confidence = True
            
        else:
            self.confidence = False
            if verbose:
                print "Center point color is {}".format(
                    image.getHSV(*self.center))
            return
        
        #NOISE_DIST = 3
        
        maxDist = searchRange + self.radius
        
        ## Find the x-coordinate of the center of the circle
        
        left = np.array((-1, 0))
        leftPoint = doBinarySearch(image, self.color, 
                       self.center, self.center - maxDist*left)
        
        right = np.array((1, 0))
        rightPoint = doBinarySearch(image, self.color, 
                       self.center, self.center - maxDist*right)
        
        self.center = np.array((leftPoint + rightPoint) / 2, dtype=np.int32)
        
        ## Find the y-coordinate of the center
        up = np.array((0, 1))
        topPoint = doBinarySearch(image, self.color, 
                       self.center, self.center - maxDist*up)
        
        down = np.array((0, -1))
        bottomPoint = doBinarySearch(image, self.color, 
                       self.center, self.center - maxDist*down)
        
        self.center = np.array((topPoint + bottomPoint) / 2, dtype=np.int32)
        
        ## Calculate the radius
        self.radius = int(np.linalg.norm(topPoint - bottomPoint) / 2)
        
        
        
        


In [139]:
green = Color((30, 60, 100), (60, 255, 255))
orange = Color((5, 150, 130), (30, 255, 255))

greenCircle = Circle2D((1920/2, 1080/2), 400, green)
orangeCircle = Circle2D((1920/2, 1080/2), 400, orange)

greenSmall = Circle2D((1100, 700), 130, green)
orangeSmall = Circle2D((600, 700), 130, orange)

# Define helper functions for searching

In [140]:
def doLinearSearch(image, color, start, end, 
                   numSteps=None, tol=0, verbose=False):
    """
    Runs a linear search between start and end, 
    both specified as iterables of length 2
    (x, y) where start must match color and end must not.
    Tolerates stretches of incorrect pixel color up to tol long without
    returning wildly incorrect results.
    
    Returns a np array of the coordinates of the transition found
    """
    end = np.array(end)
    start = np.array(start)
    if numSteps is None:
        numSteps = max(np.abs(end - start))
        
    stepsize = (end - start)/float(numSteps)
    if verbose:
        print "stepsize={}".format(stepsize)
    
    missCount = 0
    for i in range(numSteps):
        point = np.array(start + i*stepsize, dtype=np.int32)
        if not color.matches(image.getHSV(*point)):
            missCount += 1
        else:
            missCount = 0

        if missCount > tol:
            result = np.array(start + (i - tol)*stepsize, dtype=np.int32)
            if verbose:
                print "Search end found after {} steps at {}".format(i, result)
            break
            
    return result
    

In [141]:
def doBinarySearch(image, color, start, end, 
                   numSteps=float('inf'), verbose=False):
    """
    Runs a binary search between start and end, 
    both specified as iterables of length 2
    (x, y) where start must match color and end must not.
    
    Returns a np array of the coordinates of the transition found
    """
    lower = np.array(start, dtype=np.float64)
    upper = np.array(end, dtype=np.float64)
    
    i = 0
    while i < numSteps and max(np.abs(upper - lower)) > 1:
        point = (upper + lower) / 2
        
        if color.matches(image.getHSV(*point)):
            lower = point
        else:
            upper = point

            
    return np.array((upper + lower) / 2, dtype=np.int32)

## Tests

# Capture and handle video

In [142]:
FILE = '../test_data/smooth_1.mp4'
cv2.namedWindow('raw_frame')

In [144]:
cap = cv2.VideoCapture(FILE)

def handleFrame(frame, features, verbose=False):
    image = Image(frame)

    for f in features:
        f.refine(image, verbose=verbose)
    
    return image


def main():
    features = deepcopy([greenSmall, orangeSmall])
    i=0
    while cap.isOpened():
        verbose = i%50 == 0
        if verbose:
            clear_output()
        i += 1
        
        startTime = time.time()
        ret, frame = cap.read()
        readTime = time.time()

        if frame is None or len(frame) <= 0:
            print "End of video file reached"
            break

        # Do work here
        image = handleFrame(frame, features, verbose=verbose)
        
        finishTime = time.time()

        for f in features:
            f.draw(frame)
            
        image.draw(frame)
        scaled = cv2.resize(frame, dsize=None, fx=0.5, fy=0.5)
        cv2.imshow('raw_frame', scaled)
        
        drawnTime = time.time()

        cv2.waitKey(1)
        
        if verbose:
            print "{0:.1f}ms reading and drawing image, {1:.1f}ms processing, {2} pixels accessed\n".format(
                (readTime-startTime + drawnTime-finishTime)*1000, 
                (finishTime-readTime)*1000, 
                len(image.pixelsAccessed))
        
main()

7.5ms reading and drawing image, 2.8ms processing, 74 pixels accessed

End of video file reached


# Profile things

In [135]:
cap = cv2.VideoCapture(FILE)

def bench():
    features = deepcopy([greenSmall])
    for _ in range(60):
        ret, frame = cap.read()
        handleFrame(frame, features)
    
%lprun -r -f doBinarySearch bench()